In [267]:
import pandas as pd
import requests

url = "https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations=ADM%2C+BKL%2C+BMW%2C+BNK%2C+CDP%2C+DSS%2C+FOR%2C+GOL%2C+GRZ%2C+HMB%2C+IDC%2C+IDP%2C+INN%2C+LLP%2C+MED%2C+NLS%2C+PLP%2C+RRM%2C+RTL%2C+SHM%2C+SLT%2C++SNM%2C+SQV%2C+TK2%2C++ALP%2C+BLC%2C+BLK%2C+BLS%2C+BSK%2C+CAP%2C+CSL%2C+DAN%2C+DDM%2C+EBB%2C+EP5%2C+FDC%2C+FLL%2C+GIN%2C+GKS%2C+GNL%2C+HGM%2C+HOR%2C+HRS%2C+HVN%2C+HYS%2C+LBD%2C+LVM%2C+LVT%2C+MDW%2C+MNT%2C+MRL%2C+MSK%2C+PSN%2C+RBB%2C+RBP%2C+RP2%2C+SDW%2C+SIL%2C+SLI%2C+SPS%2C+SPT%2C+STR%2C+TCC%2C+TNY%2C+TUM%2C+VRG%2C+WC3%2C+WHW%2C+BCH%2C+BGP%2C+CHM%2C+CHP%2C+CRL%2C+CSV%2C+CWD%2C+DPO%2C+GNF%2C+GRM%2C+GRV%2C+KSP%2C+MTM%2C+PSC%2C+PSR%2C+RCK%2C+SLK%2C+STL%2C+SWM%2C+TMR%2C+TUN%2C+UTY%2C+VLC%2C+WTM&SensorNums=18&dur_code=D&Start=2013-01-01&End=2023-04-11"

response = requests.get(url).json()

In [268]:
station_list = []
date_list = []
value_list = []

for station in response:
    if "obsDate" in station:
        station_id = station['stationId']
        date = pd.to_datetime(station['obsDate'])
        value = station['value']
        station_list.append(station_id)
        date_list.append(date)
        value_list.append(value)

snow_df = pd.DataFrame({
    "Date" : date_list,
    "Station ID" : station_list,
    "Snow Depth (in)" : value_list,
})

snow_df["Month"] = snow_df["Date"].dt.month
snow_df["Year"] = snow_df["Date"].dt.year
snow_df.head(50)

,Date,Station ID,Snow Depth (in),Month,Year
0,2013-01-01,ADM,24,1,2013
1,2013-01-02,ADM,24,1,2013
2,2013-01-03,ADM,23,1,2013
3,2013-01-04,ADM,23,1,2013
4,2013-01-05,ADM,23,1,2013
5,2013-01-06,ADM,23,1,2013
6,2013-01-07,ADM,22,1,2013
7,2013-01-08,ADM,22,1,2013
8,2013-01-09,ADM,22,1,2013
9,2013-01-10,ADM,25,1,2013


In [233]:
# snow_yearly = snow_df.groupby(["Year"])
# winter_data = []
# # snow_yearly.head(24)

# for row in range(len(snow_yearly)):
#     data_year = snow_yearly["Year"]
#     # data_month = row[1]["Month"]
#     print(data_year)
#     # cur_year = 2013
#     # if data_year == cur_year:
#     #     if (data_month >= 10) :
#     #         winter_data.append(row[1])
#     # elif data_year != cur_year:
#     #     cur_year = data_year
                

In [276]:
winter_bin = [0,4,10,12]
labels = ["Winter_End", "Summer", "Winter_Start"]

snow_binned = snow_df.copy()
snow_binned["Season"] = pd.cut(x = snow_binned["Month"], bins =  winter_bin, labels = labels)

snow_season = snow_binned.sort_values(["Year", "Month"])

season_list = []
working_year = 2014
for row in snow_season.iterrows():
    row_year = row[1][4]
    row_season = row[1][5]

    if (row_season == "Winter_End") & (row_year == working_year):
        season_list.append(f"Winter {working_year}")
    elif (row_season == "Winter_Start") & (row_year == (working_year - 1)):
        season_list.append(f"Winter {working_year}")
    elif (row_season == "Summer"):
        season_list.append(f"Summer {working_year}")
    else:
        season_list.append(f"na")

    if (row_year == (working_year + 1)):
        working_year = working_year + 1



# snow_season["Season"].unique()
snow_season["Season"] = season_list
snow_season_grouped = snow_season.groupby(["Season"])

snow_season_grouped.mean().head(50)

,Snow Depth (in),Month,Year
Season,,,
Summer 2014,-27.220708,7.504087,2013.498638
Summer 2015,0.677596,7.502732,2015.000000
Summer 2016,0.255556,7.544444,2016.000000
Summer 2017,-0.758242,7.489011,2017.000000
Summer 2018,-0.402174,6.000000,2018.000000
Summer 2019,-2856.857143,8.000000,2019.000000
Summer 2020,4.273973,7.863014,2020.000000
Summer 2021,1.779412,6.132353,2021.000000
Summer 2022,4.350746,7.500000,2022.000000
